# Sourcery PDF Workflow Notebook

End-to-end PDF extraction and reviewer artifact generation.

In [ ]:
%pip install -q sourcery pydantic pypdf pillow pytesseract

In [ ]:
from pathlib import Path
from pydantic import BaseModel
import sourcery
from sourcery.contracts import (
    EntitySchemaSet,
    EntitySpec,
    ExtractionExample,
    ExtractionTask,
    ExampleExtraction,
    RuntimeConfig,
)


class ConceptAttrs(BaseModel):
    category: str | None = None
    note: str | None = None


task = ExtractionTask(
    instructions="Extract architecture concepts and benchmark metrics.",
    schema=EntitySchemaSet(entities=[EntitySpec(name="concept", attributes_model=ConceptAttrs)]),
    examples=[
        ExtractionExample(
            text="The Transformer uses self-attention.",
            extractions=[
                ExampleExtraction(
                    entity="concept",
                    text="self-attention",
                    attributes={"category": "architecture", "note": "core mechanism"},
                )
            ],
        )
    ],
)

pdf_path = Path("1706.03762v7.pdf")
runtime = RuntimeConfig(
    model="deepseek/deepseek-chat",
    reconciliation={"enabled": True, "use_workforce": True},
)

result = sourcery.extract_from_sources(
    pdf_path,
    task=task,
    runtime=runtime,
)

result.metrics.model_dump(mode="json")

In [ ]:
doc = result.documents[0]
len(doc.extractions), len(doc.canonical_claims)

In [ ]:
from sourcery.io import save_extract_result_jsonl, write_document_html, write_reviewer_html

output_dir = Path("examples/output/notebook_pdf")
output_dir.mkdir(parents=True, exist_ok=True)

jsonl_path = output_dir / "1706.03762v7.sourcery.jsonl"
viewer_path = output_dir / "1706.03762v7.sourcery.viewer.html"
reviewer_path = output_dir / "1706.03762v7.sourcery.reviewer.html"

save_extract_result_jsonl(result, jsonl_path)
write_document_html(doc, viewer_path)
write_reviewer_html(doc, reviewer_path)

str(jsonl_path), str(viewer_path), str(reviewer_path)